In [184]:
texts = []

with open('/kaggle/input/mahabharat-index/cleanedTextFileFinalDONE.txt', 'r',encoding='utf-8') as file:
    for line in file:
        texts.append(line[:-1])
        


In [185]:
# ! pip uninstall diaparser

In [186]:
! pip install diaparser

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [187]:
texts[20:30]

['Abhīru, a king, who was an incarnation of the sixth of the eight Asuras of the family of the Kāleyāḥ, .',
 'Abhisāra, a people  . Together with other barbarous tribes they fight Arjuna on the th day of the great battle, . They are mentioned by Kṛṣṇa among the auxiliaries of Duryodhana, on the th day of the battle.',
 'Abhīsāra, enumerated by Sañjaya among the people of Bhāratavarṣa, .',
 'Abhisārī, a town conquered by Arjuna  on his Digvijaya, .',
 'Abhiṣāha, a people  with other barbarous tribes they attack Arjuna at the instigation of Duḥśāsana on the th day of the great battle, . .',
 'Abhīṣāha, a people   first day of the battle, .  th day, .  th day, .  th day, . After the th day, .',
 'Ābhiṣecanika parva, i.e .',
 'Abhiṣyanta, the second of the five sons of Kuru and Vāhinī, .',
 'Abhivādya = Śiva',
 'Acala1, brother in law of Dhṛtāraṣṭra, brother of Śakuni, .']

In [1]:
def extract_rel_window(text_list, keywords_list):
    sents = []
    for sent in text_list: 
        for keyword in keywords_list:
            if keyword in sent:
                index = sent.find(keyword)
                min_idx = index
                max_idx = index
                
                while min_idx>=0 and sent[min_idx]!='.':
                    min_idx = min_idx-1
                while max_idx<len(sent) and sent[max_idx]!='.':
                    max_idx = max_idx+1
                
                # max_idx = len(sent)-1
                # if index+window_size<len(sent)-1:
                #     max_idx = index+window_size
                min_idx = min_idx + 1
                sents.append(sent[min_idx:max_idx])
    return sents

In [208]:
sents = extract_rel_window(texts,["daughter of"])
sents = [sent.lower() for sent in sents]
sents

[' after the elapse of the exile he joins the pāṇḍavas in upaplavya with kṛṣṇa and subhadrā, and is there married to uttarā, daughter of king virāṭa ',
 'agastya1, a ṛṣi, brother of vasiṣṭha and son of mitra and varuṇa he is called maitrāvaruṇi  and kumbhayoni   he once saw his ancestors hanging in a cave, and was urged by them to procure progeny he then from the most beautiful parts of all animals created lopāmudrā and let her become the daughter of the king of vidarbha when he had married her he, in order to procure riches for her, applied successively to three kings, and then to the dānava ilvala, who shaped his own younger brother into a he goat that was digested by agastya',
 ' as he did not see any wife worthy of himself, he selected the most beautiful parts from various creatures and created an excellent woman, whom he caused to be born as the daughter of the vidarbha king, who was then undergoing ascetic penances for obtaining offspring',
 ' the brahmansbhāvayantibeinganddaught

## Removing trailing commas and putting a full stop

In [2]:
def remove_before_last_alphabet(sentence):
    # Find the index of the last alphabet
    last_alpha_index = None
    for i in range(len(sentence) - 1, -1, -1):
        if sentence[i].isalpha():
            last_alpha_index = i
            break
    
    # Remove everything before the last alphabet
    if last_alpha_index is not None:
        sentence = sentence[0:last_alpha_index+1]
    
    # Add a full stop at the end
    sentence = sentence.strip() + "."
    
    return sentence

def assign_token_ids(sent):
    # Initialize an empty list to store token information
    tokens = []

    # Iterate over each token in the sentence
    for i, (word, head, rel) in enumerate(zip(sent.words, sent.arcs, sent.rels), start=1):
        # Extract relevant information from each token
        token_info = {
            "id": i,
            "word": word,
            "head": head,
            "rel": rel,
        }
        tokens.append(token_info)

    return tokens

import nltk
nltk.download('wordnet')

from nltk.corpus import wordnet

def is_english_word(word):
    # Check if the word is in WordNet's list of English words
    return wordnet.synsets(word) != []

def transliterate_to_latin(word):
    iast_mapping = {
        'á': 'aa',
        'ñ': 'n',
        'ā': 'aa',
        'ă': 'aa',
        'ī': 'i',
        'ĭ': 'i',
        'Ś': 'sh',
        'ś': 'sh',
        'Ū': 'u',
        'ū': 'u',
        'ḍ': 'd',
        'Ḣ': '',
        'ḥ': '',
        'ṃ': 'm',
        'ṅ': 'n',
        'ṇ': 'n',
        'ṛ': 'ri',
        'ṝ': 'ri',
        'ṣ': 'sh',
        'ṭ': 't'
    }

    latin_word = ''
    for char in word:
        if char in iast_mapping:
            latin_word += iast_mapping[char]
        else:
            latin_word += char
    return latin_word





[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Dependency parsing trials - **FATHER**

In [3]:
from diaparser.parsers import Parser

In [4]:
parser = Parser.load('en_ewt.electra-base')

# Function to extract relations

In [5]:
#TEMP
def extract_rel(sents,rel):
    ans_dict = []
    my_dict = {}
    found_relations=[]
    
    
    # sentences_10 = [ "He explained to him why kine have descended to earth In days of yore, when in the Deva yuga the Dānavas became lords of the three worlds, Aditi underwent the severest austerities and begot Viṣṇu seeing Aditi perform penances, Surabhī, the daughter of Dakṣa, also underwent austerities on Kailāsa, in yoga, standing upon one leg for, years Ṛ and M."]
    
    for sent in sents:
        if len(sent)<5:
            continue
        sent = remove_before_last_alphabet(sent)
        sent = transliterate_to_latin(sent)
        a = assign_token_ids((parser.predict(sent, text='en')).sentences[0])
        # finding head 
        head=""
        id=0
        for i in a:
#             print(i)
            if i['word']==rel:
                id  = i['id']
                head_pointer = i['head']
                for j in a:
                    if j['id']==head_pointer:
                        head = j['word']
    
        if head=="":
            continue
            
        # finding tail 
        tails = []
        tail = ""
        for i in range(id+1 , len(a)):
            if a[i]['rel'] != 'nmod' or a[i]['word'] == 'king' or a[i]['word'] == 'queen':
                tail = tail + a[i]['word'] + " "
            else:
                tails.append(tail+a[i]['word'])
                j=i+1
                #find multiple tails if possible:
                while(j<len(a) and (a[j]['word'] in ['and',','] or a[j]['rel'] == 'conj')):
                    if a[j]['word']=='and' or a[j]['word']==',':
                        del a[j]
                    elif a[j]['rel'] == 'conj':
                        tails.append(a[j]['word'])
                        j=j+1
                    
                break

#         print(tails)
        tails = [tail for tail in tails if len(tail.split()) < 6]
        
        # print(head , 'father ', tail , 'text : ', sent,'\n')
        # found_relations.append(head + ' father ' + tail )

        for tail in tails:
            my_dict = {}
            my_dict['head']=head
            my_dict['relation']=rel+' of'
            my_dict['tail']=tail
            my_dict['text']=sent
            ans_dict.append(my_dict)
            
    
    return ans_dict    
    
    
# printing answers

# for i in found_relations:
#     print(i)


    
    
# print(found_relations)
# print(ans_dict)

In [211]:
len(sents)

120

In [212]:
rels = extract_rel(sents,"daughter")

In [215]:
# rels
for i in rels:
    print(i['text'],'\n')
    print(i['head'] , i['relation'] , i['tail'],'\n')
    

after the elapse of the exile he joins the paandavas in upaplavya with krishna and subhadraa, and is there married to uttaraa, daughter of king viraata. 

uttaraa daughter of king viraata 

agastya1, a rishi, brother of vasishtha and son of mitra and varuna he is called maitraavaruni  and kumbhayoni   he once saw his ancestors hanging in a cave, and was urged by them to procure progeny he then from the most beautiful parts of all animals created lopaamudraa and let her become the daughter of the king of vidarbha when he had married her he, in order to procure riches for her, applied successively to three kings, and then to the daanava ilvala, who shaped his own younger brother into a he goat that was digested by agastya. 

let daughter of the king of vidarbha 

ambaa, the eldest daughter of the king of kaashi, chosen by the saubha king, carried off by bhishma to be married to vicitravirya, but released refused by the saubha king, etc reborn as shikhandin. 

ambaa daughter of the king o

In [217]:
import pickle

In [219]:
with open('DAUGHTER.pkl', 'wb') as f:
    pickle.dump(rels, f)

## Mother

In [262]:
sents_mother = extract_rel_window(texts,["mother of"])
sents_mother = [sent.lower() for sent in sents_mother]


In [263]:
print(len(sents_mother))

47


In [264]:
print(sents_mother)

['aśmakī, a yādavī, the wife of king prācinvat and mother of saṃyāti, ', 'aditi1, one of the thirteen daughters of dakṣa, who were married to kaśyapa by him she is mother of the twelve ādityas               ', 'adṛśyantī, wife of śaktri  and daughter in law of vasiṣṭha, and mother of parāśara   ', 'aikṣvākī, a princess, wife of suhotra, and mother of ajamīḍha  ', 'āṅgī, an ancient queen, wife of ariha and mother of mahābhauma ', 'citrāṅgadā, daughter of citravāhana and mother of babhru vāhana    in the encounter between arjuna and babhruvāhana both fell n in a swoon', 'śacī, the queen of indra, daughter of puloman and mother of jayanta           ', 'śakuntalā, daughter of viśvāmitra and menakā, wife of duṣyanta, and mother of bharata      ', 'dākṣāyaṇī = the mother of rājadharman ', 'damayantī, daughter of the vidarbha king bhīma, wife of nala and mother of indrasena and indrasenā        ', 'danu, daughter of dakṣa and wife of kaśyapa, mother of the dānavas        ', 'devakī, daughter 

In [266]:
rels_mother = extract_rel(sents_mother,"mother")

In [267]:
sents_m = ["shaci, the queen of indra, daughter of puloman , mother of jayanta."]
extract_rel(sents_m,"mother")

[{'head': 'shaci',
  'relation': 'mother of',
  'tail': 'jayanta',
  'text': 'shaci, the queen of indra, daughter of puloman , mother of jayanta.'}]

In [7]:
sents_m = ["Shaci, the queen of Indra, daughter of Puloman, mother of Jayanta."]
extract_rel(sents_m,"mother")

[{'head': 'Shaci',
  'relation': 'mother of',
  'tail': 'Jayanta',
  'text': 'Shaci, the queen of Indra, daughter of Puloman, mother of Jayanta.'}]

In [269]:
# extract_rel(sents_mother,"mother")

In [270]:
# rels
print(len(rels_mother))
for i in rels_mother:
    print(i['text'],'\n')
    print(i['head'] , i['relation'] , i['tail'],'\n')



50
ashmaki, a yaadavi, the wife of king praacinvat and mother of samyaati. 

wife mother of samyaati 

adrishyanti, wife of shaktri  and daughter in law of vasishtha, and mother of paraashara. 

adrishyanti mother of paraashara 

aikshvaaki, a princess, wife of suhotra, and mother of ajamidha. 

princess mother of ajamidha 

aangi, an ancient queen, wife of ariha and mother of mahaabhauma. 

queen mother of mahaabhauma 

citraangadaa, daughter of citravaahana and mother of babhru vaahana    in the encounter between arjuna and babhruvaahana both fell n in a swoon. 

daughter mother of babhru vaahana 

shaci, the queen of indra, daughter of puloman and mother of jayanta. 

daughter mother of jayanta 

shakuntalaa, daughter of vishvaamitra and menakaa, wife of dushyanta, and mother of bharata. 

shakuntalaa mother of bharata 

damayanti, daughter of the vidarbha king bhima, wife of nala and mother of indrasena and indrasenaa. 

damayanti mother of indrasena 

damayanti, daughter of the vi

In [272]:
with open('MOTHER.pkl', 'wb') as f:
    pickle.dump(rels_mother, f)

## Sister

In [273]:
sents_sister = extract_rel_window(texts,["sister of"])
sents_sister = [sent.lower() for sent in sents_sister]


In [274]:
print(len(sents_sister))

16


In [275]:
print(sents_sister)

['abhimanyu, son of arjuna and subhadrā, sister of kṛṣṇa, is an incarna tion of varcas, son of soma, wherefore he enters soma after death, after his karman has elapsed   etymology of his name  he is the vaṃśakara of the pāṇḍavas ', 'ambikā, sister of ambālikā      ', 'the th of the minor parvans sauti related how the ascetic jaratkāru, upon a woman, likewise named jaratkāru, being the sister of the serpent king vāsuki, begot a son  destined to neutralize the curse inflicted on the serpents by their mother  origin of the serpents and of garuḍa and aruṇa in order to explain the origin of uccaiḥśravas, sauti related the churning of the ocean and the consequent production of the amṛta, etc kadrū and vinatā made a bet as to the colour of the tail of uccaiḥśravas, and vinatā  became the slave of kadrū kadrū cursed the serpents, her sons, to be burnt at the snake sacrifice of janamejaya exploits of garuḍa aruṇa became the charioteer of the  sun garuḍa set out to bring the amṛta to the snakes,

In [276]:
rels_sister = extract_rel(sents_sister,"sister")

In [277]:
# rels
print(len(rels_sister))
for i in rels_sister:
    print(i['text'],'\n')
    print(i['head'] , i['relation'] , i['tail'],'\n')



17
abhimanyu, son of arjuna and subhadraa, sister of krishna, is an incarna tion of varcas, son of soma, wherefore he enters soma after death, after his karman has elapsed   etymology of his name  he is the vamshakara of the paandavas. 

subhadraa sister of krishna 

ambikaa, sister of ambaalikaa. 

ambikaa sister of ambaalikaa 

the th of the minor parvans sauti related how the ascetic jaratkaaru, upon a woman, likewise named jaratkaaru, being the sister of the serpent king vaasuki, begot a son  destined to neutralize the curse inflicted on the serpents by their mother  origin of the serpents and of garuda and aruna in order to explain the origin of uccaishravas, sauti related the churning of the ocean and the consequent production of the amrita, etc kadru and vinataa made a bet as to the colour of the tail of uccaishravas, and vinataa  became the slave of kadru kadru cursed the serpents, her sons, to be burnt at the snake sacrifice of janamejaya exploits of garuda aruna became the ch

In [278]:
with open('SISTER.pkl', 'wb') as f:
    pickle.dump(rels_sister, f)